In [1]:

import sys, os
# # add ../ to sys.path
# sys.path.append('../')

# set CUDA_VISIBLE_DEVICES
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

# change current directory
os.chdir('../')

import argparse
import gc
import random
import torch
from PIL import Image
from torchvision.transforms import PILToTensor
from icm.src.models.dift_sd import SDFeaturizer
from icm.src.utils.visualization import Demo

/home/guohe/anaconda3/envs/icm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dift = SDFeaturizer()

In [ ]:
# you can choose visualize cat or guitar
category = 'cat'

print(f"let's visualize semantic correspondence on {category}")

In [ ]:
if category == 'cat':
    filelist = ['datasets/example_cats/image/cat1.png', 'datasets/example_cats/image/cat2.jpg']
elif category == 'guitar':
    filelist = ['./assets/guitar.png', './assets/target_guitar.png']

prompt = f'a photo of a {category}'

In [ ]:
ft = []
imglist = []

# decrease these two if you don't have enough RAM or GPU memory
img_size = 768
ensemble_size = 8

for filename in filelist:
    img = Image.open(filename).convert('RGB')
    img = img.resize((img_size, img_size))
    imglist.append(img)
    img_tensor = (PILToTensor()(img) / 255.0 - 0.5) * 2
    ft.append(dift.forward(img_tensor,
                           prompt=prompt,
                           ensemble_size=ensemble_size,
                           up_ft_index = 3))
ft = torch.cat(ft, dim=0)

gc.collect()
torch.cuda.empty_cache()

In [ ]:
demo = Demo(imglist, ft, img_size)

In [ ]:
'''
left is source image, right is target image.
you can click on the source image, and DIFT will find the corresponding
point on the right image, mark it with red point and also plot the per-pixel 
cosine distance as heatmap.
'''
demo.plot_img_pairs(fig_size=5)